In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_path="Dataset/en-de/"
en = pd.read_csv(base_path+'idiom_trainplus.en',sep='\n', header=None,names=['en'])
de = pd.read_csv(base_path+'idiom_trainplus.de',sep='\n', header=None,names=['de'])
info = pd.read_csv(base_path+'idiom_trainplus.info',sep='\t',header=None,names=["spread", "en_idiom", "de_idiom",'frequency'])
data = pd.concat([en,de,info],axis=1)
data.columns = ['en','de',"spread", "en_idiom", "de_idiom",'frequency']
data = data.drop(['spread','frequency'],axis=1)
data['en_task'] = 'translate to german: '+data['en']

In [3]:
data

,en,de,en_idiom,de_idiom,en_task
0,Could you please pass this request on to the P...,Könnten Sie diese Frage bitte an den Ratsvorsi...,to pass on,das zeitliche segnen,translate to german: Could you please pass thi...
1,"You may be forbidden to pass it on , and you a...","Es ist Ihnen vielleicht verboten , sie weiterz...",to pass on,das zeitliche segnen,translate to german: You may be forbidden to p...
2,I should like to thank everyone who has helped...,"Ich glaube , man darf hier allen , die mitgeho...",to pull together,am gleichen strang ziehen,translate to german: I should like to thank ev...
3,I will pass on your appreciation to my colleag...,Ich werde Ihre anerkennenden Worte an Gay Mitc...,to pass on,das zeitliche segnen,translate to german: I will pass on your appre...
4,That is why our group is calling for the Confe...,"Darum fordert unsere Fraktion , dass die Entsc...",to be on hold,auf eis liegen,translate to german: That is why our group is ...
...,...,...,...,...,...
2297,Popularized by Joan Jett ( originally by The A...,Anfangs war ich diesem Album gegenüber etwas s...,i say !,hören sie mal !,translate to german: Popularized by Joan Jett ...
2298,"Today &apos;s debate is , therefore , very imp...",Die heutige Debatte ist daher insofern sehr be...,to pass on,das zeitliche segnen,"translate to german: Today &apos;s debate is ,..."
2299,"At the end of April 2010 , they came together ...",Sie versammelten sich Ende April 2010 am Klaus...,to pass on,das zeitliche segnen,translate to german: At the end of April 2010 ...
2300,Our reception is open 24 hours for your conven...,In unseren Konferenzräumen finden bis zu 80 Pe...,to go places,es zu etwas bringen,translate to german: Our reception is open 24 ...


### Skip preprocessing

### Load model and get predictions

In [4]:
from transformers import TFT5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [12]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [6]:
article = data['en_task'].to_list()
article_batches = []
[article_batches.append(b) for b in batch(article, 50) ]
translation = []
#have to use a for loop because else the kernel dies
for art in tqdm(article_batches):
    batch = tokenizer.prepare_seq2seq_batch(src_texts=art, return_tensors="tf")
    #print("batch prepare done, start generating output id's")
    output_ids = model.generate(input_ids=batch.input_ids, num_return_sequences=1, num_beams=8, length_penalty=0.1)
    #print("output id's generated, starting decoding")
    translation.extend(tokenizer.batch_decode(output_ids))

100%|██████████| 47/47 [17:24<00:00, 22.23s/it]


In [7]:
data['de_translation_t5'] = translation

In [8]:
model = TFT5ForConditionalGeneration.from_pretrained('outputs_training/model_files/pytorch_model.bin',from_pt=True,config='outputs_training/model_files/config.json')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'lm_head.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [13]:
article = data['en_task'].to_list()
article_batches = []
[article_batches.append(b) for b in batch(article, 50) ]
translation = []
#have to use a for loop because else the kernel dies
for art in tqdm(article_batches):
    batch = tokenizer.prepare_seq2seq_batch(src_texts=art, return_tensors="tf")
    #print("batch prepare done, start generating output id's")
    output_ids = model.generate(input_ids=batch.input_ids, num_return_sequences=1, num_beams=8, length_penalty=0.1)
    #print("output id's generated, starting decoding")
    translation.extend(tokenizer.batch_decode(output_ids))

100%|██████████| 47/47 [16:28<00:00, 21.04s/it]


In [14]:
data['de_translation_t5_trained'] = translation

In [15]:
data.to_csv('preliminary_results.csv')

In [16]:
#not sure if we even have to load in the tokenizes, is it even trained? idk
tokenizer = T5Tokenizer.from_pretrained('outputs_training/model_files/pytorch_model.bin',from_pt=True,config='outputs_training/model_files/tokenizer_config.json')

RuntimeError: Internal: C:\projects\sentencepiece\src\sentencepiece_processor.cc(891) [model_proto->ParseFromArray(serialized.data(), serialized.size())] 